In [1]:
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from aicsimageio.types import PhysicalPixelSizes
from pathlib import Path
import numpy as np

In [2]:
base_path = "/mnt/eternus/share/immun_project/"
cut_path = base_path / Path("cut")
new_path = base_path / Path("source_cut")

filenames = sorted(cut_path.glob("*_DAPI.tif"))

In [3]:
for fn in filenames:
    fn_base = fn.stem[:-5]
    dapi = AICSImage(fn).get_image_data("YX", Z=0, T=0, C=0)
    CD64 = AICSImage(cut_path / f"{fn_base}_CD64.tif").get_image_data("YX", Z=0, T=0, C=0)
    TMEM119 = AICSImage(cut_path / f"{fn_base}_TMEM119.tif").get_image_data("YX", Z=0, T=0, C=0)
    merged = np.stack([dapi, CD64, TMEM119], axis=0)
    out_fn = new_path / f"{fn_base}.tiff"
    OmeTiffWriter.save(merged, out_fn, dim_order="CYX", physical_pixel_sizes=[PhysicalPixelSizes(0, 0.325, 0.325)], channel_names=[["DAPI", "CD64", "TMEM119"]])

/mnt/eternus/anaconda3_shared/envs/jc_im2im_dev/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
/mnt/eternus/anaconda3_shared/envs/jc_im2im_dev/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
/mnt/eternus/anaconda3_shared/envs/jc_im2im_dev/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The de

In [4]:
reader = AICSImage(out_fn)
print(reader.physical_pixel_sizes.Y)
print(reader.physical_pixel_sizes.X)
print(reader.dims)

0.325
0.325
<Dimensions [T: 1, C: 3, Z: 1, Y: 2600, X: 2600]>


In [5]:
img_DAPI = reader.get_image_data("YX", C=reader.channel_names.index("DAPI"), T=0, Z=0)
img_CD64 = reader.get_image_data("YX", C=reader.channel_names.index("CD64"), T=0, Z=0)
img_TMEM119 = reader.get_image_data("YX", C=reader.channel_names.index("TMEM119"), T=0, Z=0)